# Imports

In [3]:
# Third-Party Imports
import nltk
import torch

# Standard Library Imports
import os
from math import inf
from string import punctuation
import sys

from queries import get_text_cli
from get_documents import search

# Utility Functions

In [4]:
def get_text():
    term = get_text_cli('Enter a search term')
    return search(term)

def load_docs(dirname='corpus'):
    corpus = dict()
    main_path = os.path.join(os.path.dirname('__file__'), dirname)

    for file in os.listdir(main_path):
        with open(os.path.join(main_path, file), 'r') as f:
            corpus[file] = f.read()
    
    return corpus

# Simple Extractive Text Summarisation

In [5]:
def word_tokenize(text):
    banned = list(punctuation) + nltk.corpus.stopwords.words("english")
    return [w for w in nltk.word_tokenize(text) if w not in banned]

def SE_summary(text):
    # Create word and sentence tokens
    words = word_tokenize(text)
    word_set = set(words) # set of all unique words in word tokens
    sents = nltk.sent_tokenize(text)
    
    # Initialise frequency table for word tokens
    w_freq_table = {w: words.count(w) for w in word_set}
    
    # Score sentences based on frequency of their words
    sent_scores = {
        sent: sum(w_freq_table.get(w, 0) for w in word_tokenize(sent))
        for sent in sents
    }
    
    # Build summary
    avg = sum(sent_scores.values()) / len(sent_scores)
    summary = " ".join(sent for sent in sents if sent_scores[sent] >= avg)
    return summary

In [7]:
text = get_text()
text

Enter a search term: Paris


('Q90',
 'Paris',
 'Paris (French pronunciation:\xa0\u200b[paʁi] (listen)) is the capital and most populous city of France, with an estimated population of 2,165,423 residents in 2019 in an area of more than 105\xa0km² (41 sq mi), making it the 30th most densely populated city in the world in 2020. Since the 17th century, Paris has been one of the world\'s major centres of finance, diplomacy, commerce, fashion, gastronomy, and science. For its leading role in the arts and sciences, as well as its very early system of street lighting, in the 19th century it became known as "the City of Light". Like London, prior to the Second World War, it was also sometimes called the capital of the world.\nThe City of Paris is the centre of the Île-de-France region, or Paris Region, with an estimated population of 12,262,544 in 2019, or about 19% of the population of France, making the region France\'s primate city. The Paris Region had a GDP of €739 billion ($743 billion) in 2019, which is the highes

In [8]:
summary = SE_summary(text[2])
summary

'Paris (French pronunciation:\xa0\u200b[paʁi] (listen)) is the capital and most populous city of France, with an estimated population of 2,165,423 residents in 2019 in an area of more than 105\xa0km² (41 sq mi), making it the 30th most densely populated city in the world in 2020. Since the 17th century, Paris has been one of the world\'s major centres of finance, diplomacy, commerce, fashion, gastronomy, and science. The City of Paris is the centre of the Île-de-France region, or Paris Region, with an estimated population of 12,262,544 in 2019, or about 19% of the population of France, making the region France\'s primate city. The Paris Region had a GDP of €739 billion ($743 billion) in 2019, which is the highest in Europe. According to the Economist Intelligence Unit Worldwide Cost of Living Survey, in 2022, Paris was the city with the ninth-highest cost of living in the world. Paris is a major railway, highway, and air-transport hub served by two international airports: Charles de Ga

In [9]:
print(len(text[2]), len(summary))

106363 56092


# Clever Algorithms

In [11]:
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

ModuleNotFoundError: No module named 'sumy'

## LexRank, TextRank, Latent Semantic Analysis (LSA), Luhn's Algorithm

In [122]:
def algo_summary(text, method):
    # Initialise summariser
    if method == "lex":
        summariser = LexRankSummarizer()
    elif method == "text":
        summariser = TextRankSummarizer()
    elif method == "lsa":
        summariser = LsaSummarizer()
    elif method == "luhn":
        summariser = LuhnSummarizer()
    
    # Initialise parser
    parser = PlaintextParser(text, Tokenizer("english"))
    
    # Create summary
    summary_sents = summariser(parser.document, 20)
    
    return " ".join(str(s) for s in summary_sents)

In [123]:
text = get_text()
text

Enter a search term: Lionel Messi


('Q615',
 'Lionel Messi',
 'Argentine professional footballer\nEponyms and public art\nFilms\nFamily\nLionel Andrés Messi[note 1] (Spanish pronunciation:\xa0[ljoˈnel anˈdɾes ˈmesi] (listen); born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for  Ligue 1 club Paris Saint-Germain and captains the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record seven Ballon d\'Or awards,[note 2] a record six European Golden Shoes, and in 2020 was named to the Ballon d\'Or Dream Team. Until leaving the club in 2021, he had spent his entire professional career with Barcelona, where he won a club-record 35 trophies, including 10 La Liga titles, seven Copa del Rey titles and four UEFA Champions Leagues. With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals in La Liga (474), most hat-trick

In [124]:
for meth in ("lex", "text", "lsa", "luhn"):
    summary = algo_summary(text[2], meth)
    print(f"\nMethod: {meth}", summary, f"Orginal: {len(text[2])}, Summary: {len(summary)}", sep="\n\n")
    print(f"\n{100*'='}\n")


Method: lex

Messi has scored over 795 senior career goals for club and country, and has the most goals by a player for a single club (672). The following two seasons, Messi finished second for the Ballon d'Or behind Cristiano Ronaldo (his perceived career rival), before regaining his best form during the 2014–15 campaign, becoming the all-time top scorer in La Liga and leading Barcelona to a historic second treble, after which he was awarded a fifth Ballon d'Or in 2015. He scored all of his side's four goals in the Champions League quarter-final against Arsenal on 6 April while becoming Barcelona's all-time top scorer in the competition. After Barcelona lost the Copa del Rey final four days later, Messi scored both goals in his side's 2–0 win in the first leg of the Champions League semi-finals in Madrid, the second of which – a slaloming dribble past three Real players – was acclaimed as one of the best ever in the competition. At the close of the year, Messi had scored a record 91 


Method: lsa

As his father's health insurance covered only two years of growth hormone treatment, which cost at least $1,000 per month, Newell's agreed to contribute, but later reneged on their promise. French winger Ludovic Giuly explained how Messi caught the eye in a training session with Frank Rijkaard's first team: "He destroyed us all... They were kicking him all over the place to avoid being ridiculed by this kid, he just got up and kept on playing. However, he continued to be plagued by major injuries; a metatarsal fracture sustained on 12 November 2006 kept him out of action for three months. Messi helped the team achieve 16 consecutive league victories, a record in Spanish football, concluding with another hat-trick against Atlético Madrid on 5 February 2011. In Barça's last home league match on 5 May, against Espanyol, Messi scored all four goals before approaching the bench to embrace Guardiola, who had announced his resignation as manager. On 17 April, Messi ended a five-